# 20 属性描述符

描述符是实现了特定协议的类，这个协议包括__get__、__set__和__delete__方法。property类实现了完整的描述符协议。

实现了__get__、__set__或__delete__方法的类是描述符。描述符的用法是，创建一个实例，作为另一个类的类属性。

<img src="Chapter20/IMG_E2D17AE68F3D-1.jpeg" alt="drawing" style="width:640px;"/>

## 20.1  描述符示例：验证属性

### 20.1.1 一个简单的描述符 —— 使用基本的描述符类创建属性

In [26]:
# 基本用法

class Quantity: # 描述符类

    def __init__(self, storage_name):
        self.storage_name = storage_name

    def __set__(self, instance, value):
        if value > 0 :
            instance.__dict__[self.storage_name] = value #  这里，必须直接处理托管实例的__dict__属性；如果使用内置的setattr函数，会再次触发__set__方法，导致无限递归。其原因就是托管属性与描述符实例名字相同！
        else:
            raise ValueError('value must be > 0!')

class LineItem: # 托管类
    weight = Quantity('weight') # 描述符实例
    price = Quantity('price')

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight # 托管属性
        self.price = price

    def subtotal(self):
        return self.price * self.weight




apples = LineItem('apple', 2.0, 3.5) # 托管实例

#print(apples.subtotal())

apples.weight = 10.0  # 存储属性


print(apples.subtotal())


print(type(apples.weight))

print(vars(apples))

35.0
<class 'float'>
{'description': 'apple', 'weight': 10.0, 'price': 3.5}


编写__set__方法时，要记住self和instance参数的意思：self是描述符实例，instance是托管实例。管理实例属性的描述符应该把值存储在托管实例中。因此，Python才为描述符中的那个方法提供了instance参数。

```py
# 正确写法：
instance.__dict__[self.storage_name] = value
# 错误写法：
self.__dict__[self.storage_name] = value
```

为了理解错误的原因，可以想想__set__方法前两个参数（self和instance）的意思。这里，self是描述符实例，它其实是托管类的类属性。同一时刻，内存中可能有几千个LineItem实例，不过只会有两个描述符实例：LineItem.weight和LineItem.price。因此，存储在描述符实例中的数据，其实会变成LineItem类的类属性，从而由全部LineItem实例共享。

### 20.1.2 变得更加复杂 —— 自动获取储存属性的名称


**为了避免在描述符声明语句中重复输入属性名，我们将为每个Quantity实例的storage_name属性生成一个独一无二的字符串。**

In [17]:
# 基本用法

class Quantity: # 描述符类
    __count = 0
    def __init__(self):
        cls = self.__class__
        index = cls.__count
        name = cls.__name__
        self.storage_name = '{}#{}'.format(name, index)
        cls.__count += 1

    def __set__(self, instance, value):
        if value > 0 :
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0!')

    def __get__(self, instance, name):
        return getattr(instance, self.storage_name)

class LineItem: # 托管类
    weight = Quantity() # 描述符实例
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight # 托管属性
        self.price = price

    def subtotal(self):
        return self.price * self.weight




apples = LineItem('apple', 2.0, 3.5) # 托管实例
banana = LineItem('banana', 1.5, 2) # 托管实例

print(apples.subtotal())

apples.weight = 10.0  # 存储属性


print(apples.subtotal())


print(vars(apples))
print(vars(banana))


7.0
35.0
{'description': 'apple', 'Quantity#0': 10.0, 'Quantity#1': 3.5}
{'description': 'banana', 'Quantity#0': 1.5, 'Quantity#1': 2}
